## 03 - Cleaning the data

This notebook continues the data cleaning from notebook `02-scapring_html`. It uses regular expressions to parce through the html bits and extract the wanted information.

In [1]:
import pandas as pd
import re

df = pd.read_csv("nominations.csv")

In [2]:
# Strip the concress_no column of any text and years. 
df['congress_no'] =  df['congress_no'].apply(lambda x: re.sub(r'th Congress .\d{4}-\d{4}.','', str(x)))

In [3]:
df['circuit'] = df['post'].str.extract('(the \w.+ Circuit)', expand=False).str.strip()

# And strip the column of any additional text so only the circuit names remain.
df['circuit'] = df['circuit'].str.replace('the District of Columbia, to be United States Circuit Judge for ','')
df['circuit'] = df['circuit'].str.replace('the ','')
df['circuit'] = df['circuit'].str.replace(' Circuit','')

In [4]:
df['confirmation_date'] = df['vote'].str.extract('(\d\d\W\d\d\W\d{4})', expand=False).str.strip()
df.tail(5)

,name,nomination_no,congress_no,post,vote,circuit,confirmation_date
189,Terrence L. O'Brien,PN882,107,"Terrence L. O'Brien, of Wyoming, to be United ...",Latest Action: 04/15/2002 - Confirmed by the S...,Tenth,04/15/2002
190,Michael J. Melloy,PN881,107,"Michael J. Melloy, of Iowa, to be United State...",Latest Action: 02/11/2002 - Confirmed by the S...,Eighth,02/11/2002
191,Michael W. McConnell,PN880,107,"Michael W. McConnell, of Utah, to be United St...",Latest Action: 11/15/2002 - Confirmed by the S...,Tenth,11/15/2002
192,William J. Riley,PN458,107,"William J. Riley, of Nebraska, to be United St...",Latest Action: 08/02/2001 - Confirmed by the S...,Eighth,08/02/2001
193,Roger L. Gregory,PN402,107,"Roger L. Gregory, of Virginia, to be United St...",Latest Action: 07/20/2001 - Confirmed by the S...,Fourth,07/20/2001


In [5]:
# I make a list out of the column vote so I can use regex on the information in the column. 
list_vote = df.vote.tolist()

In [6]:
for list in list_vote:
    try:
        sub_regex = r"Latest Action: \d\d\W\d\d\W\d{4}.*(\d\d) - (\d*).*: (\d+)[.].*"
        df.vote.str.split(sub_regex,expand=True)
    except:
        pass

votes = df.vote.str.split(sub_regex,expand=True)

In [7]:
df["yea_votes"]= votes[1]
df["nay_votes"]= votes[2]
df["record_vote_number"]= votes[3]

In [8]:
data = df.drop(['post', 'vote'], axis=1)

data.to_csv("votes.csv", index=False)